In [1]:
import tensorflow as tf
from tensorflow.keras import layers

## 循环神经网络
自然界的信号除了具有空间维度之外，还有一个时间(Temporal)维度。具有时间维度的信号非常常见，比如我们正在阅读的文本，说话时发出的语音信号，随着时间变化的股市参数等等。 这类数据并不一定具有局部相关性，同时数据在时间维度上的长度也是可变的，并不适合卷积神经网络处理。

循环神经网络可以较好的解决此类问题。

### 1. 序列表示方法
我们已经知道神经网络本质上是一系列的矩阵相乘、相加等运算，他并不能够直接处理字符串类型的数据。如果希望神经网络能够用于自然语言处理任务，那么怎么把单词或字符转化为数值就变得尤为关键。

#### one-hot
对于一个含有 n 个单词的句子， 每个单词的一种简单表示方法就是前面我们介绍的one-hot 编码。 以英文句子为例， 假设我们只考虑最常用的 10,000 个单词，那么一个单词就可以表示为某位为 1，其他位置为 0 的长度为 10,000 的稀疏向量；对于中文句子，我们也只考虑最常用的 3,500 个汉字，同样的方法，一个汉字可以用长度为 3,500 的 One-hot 向量表示。

我们把文字编码为数值的算法叫做 Word embedding， 它也表示经过这个过程后得到的词向量，具体表示算法还是词向量需要依语境定。 one-hot 的编码方式实现 Word embedding简单直观，编码过程不需要学习和训练。但是 one-hot 编码的向量是高维度(High dimensional)而且极其稀疏(Sparse)，大量的位置为 0， 计算效率较低，同时也不利于神经网络的训练。从语义角度来讲， one-hot 编码还有一个严重的问题， 它忽略了单词先天具有的语义相关性。

举个例子，对于单词“like”，“dislike”，“Rome”，“Paris” 来说，“like”和“dislike”在语义角度就强相关，他们都表示喜欢这个概念； “Rome”和“Paris”同样也是强相关，他们都表示欧洲的两个地点。 对于一组这样的单词来说，如果采用 one-hot 编码，得到的向量没有数值关系， 并不能够很好的体现相关度，因此 one-hot 编码具有明显的缺陷。

#### 余弦相关度
在自然语言处理领域，有专门的一个研究方向去探索如何学习到单词的表示向量(Word vector)，使得语义层面的相关性能够很好地在 Word Vector 上面体现出来。一个衡量表示向量的尺度就是余弦相关度(Cosine similarity):

### 2. Embedding层
Embedding 层是可训练的，他可放置在神经网络之前，完成单词到向量的转换，得到的表示向量可以继续通过神经网络完成后续任务，并计算误差ℒ，采用梯度下降算法来实现端到端(end-to-end)的训练。

Embedding层负责把单词编码为某个向量vec，它接受的是采用数字编码的单词idx

In [ ]:
# Embedding层
# 创建10个单词
x = tf.range(10)
x = tf.random.shuffle(x)

# 创建Embedding层，第一个数字表示共有多少个单词
# 第二个数字表示每个单词用多长的向量表示
net = layers.Embedding(10, 2)
out = net(x)
out

Embedding 层的查询表是随机初始化的， 需要从零开始训练。 实际上， 我们可以使用预训练的 Word embedding 模型来得到单词的表示方法，基于预训练模型的词向量相当于迁移了整个语义空间的知识，往往能得到更好的性能。

目前应用的比较广泛的预训练模型有 Word2Vec 和 GloVe 等。 他们已经在海量语料库训练得到了较好的表示方法，并可以直接导出学习到的词向量，方便迁移到其他任务。

In [ ]:
# 直接使用预训练模型
# embed_glove = tf.load_embed('glove.6B.50d.txt')

# 直接利用预训练的词向量表示初始化embedding层
# net.set_weights([embed_glove])

# 设置为不参与训练
# net.trainable = False

### 3. 循环神经网络
情感分类任务通过分析给出的文本序列， 提炼出文本数据表达的整体语义特征，从而预测输入文本的情感类型： 正面评价或者负面评价。从分类角度来看，情感分类问题就是一个简单的二分类问题，与图片分类不一样的是，由于输入是文本数据， 传统的卷积神经网络并不能取得很好的效果。那么什么类型的网络擅长处理序列数据呢？

#### 全连接层
各个单词的词向量通过 s 个全连接层分类网络 1 提取每个单词的特征，所有单词的特征最后合并，并通过分类网络 2 输出序列的类别概率分布，对于长度为 s 的句子来说， 共需要 s 个全网络层。

这 s 个全连接层共同组成了神经网络的一层，如果对接分类网络 1 共 L 层的话， 那么整个网络共包含了L ∗ s + 1个全连接层。这种方案的缺点有：
- 这种方案的网络参数量是相当可观的， 内存占用和计算代价较高，同时由于每个序列的长度 s 并不相同，网络结构是动态变化的。
- 每个全连接层子网络Wi, bi只能感受当前某个单词向量的输入，并不能感知前面的单词语境和后面单词的语境信息， 导致句子整体语义的丢失，每个子网络只能根据自己的输入来提取高层特征。

##### 共享权值
在处理序列信号的问题上，能否借鉴权值共享的思想？

s 个全连接层的网络并没有实现同享。 我们尝试将这 s 个网络层参数共享，这样其实相当于使用一个全连接网络来提取所有单词的特征信息。

通过权值共享后，参数量大大减少， 网络训练变得更加高效。但是，这种网络结构还是将整个句子拆开来分布理解，无法获取整体的语义信息。

##### 语义信息
如何赋予网络提取整体语义特征的能力呢？ 或者说，如何让网络能够按序提取词向量的语义信息，并累积成整个句子的语境信息呢？我们想到了内存(Memory)机制。如果网络能够提供一个单独的 memory 变量，每次提取词向量的特征并刷新 memory，直至最后一个输入完成，此时的 memory 即存储了所有序列的语义特征，并且由于输入之间的先后顺序，使得 memory 内容与序列顺序紧密关联。

我们将上述memory机制实现为一个状态张量，除了原来的权值共享外，还额外增加了一个参数Whh，这个新增加的参数与状态张量有关，初始化状态张量为h0，经过s个词向量的输入后得到网络最终的状态张量hs，hs较好地代表了句子地语义信息，基于hs通过某个全连接层分类器可以完成情感分类。

#### 循环神经网络
通过一步步探索，我们最终提出了一种“新型” 的网络结构，在每个时间戳，网络层接受当前时间戳的输入𝒙t和上一个时间戳的网络状态向量 h(t−1)。

运算后得到当前时间戳的新状态向量 h𝑡。

我们把时间戳上折叠，则形成循环神经网络，即首先输入一个词向量，经过运算后形成一个输出，并且更新内部的状态张量（即产生循环），接着输入第二个词向量，经过运算后形成第二个输出，并再次更新内部的状态张量（第二次循环）。

循环神经网络中一般使用tanh激活函数，并且可以选择不适用偏置b来进一步减少参数量。状态向量ht可以直接用作输出，也可以对ht做一个线性变换后得到每个时间戳上的网络输出ot。

#### 梯度传播
为了计算Whh的梯度，因为它被每个时间戳i上权值共享，我们计算Whh的梯度的时候需要将每个中间时间戳i上面的梯度求和。

### 4. RNN层的实现
基础循环神经网络，他一般叫做 SimpleRNN。 SimpleRNN 与 SimpleRNNCell 的区别在于，带 Cell 的层仅仅是完成了一个时间戳的前向运算，不带 Cell 的层一般是基于Cell 层实现的，他在内部已经完成了多个时间戳的循环运算，因此使用起来更为方便快捷。

#### SimpleRNNCell

In [ ]:
cell = layers.SimpleRNNCell(3)
cell.build(input_shape = (None, 4))
cell.trainable_variables

SimpleRNNCell 内部维护了 3 个张量， kernel 变量即Wxh张量， recurrent_kernel变量即 张量Whh， bias 变量即偏置𝒃向量。 但是 RNN 的 Memory 向量 并不由SimpleRNNCell 维护，需要用户自行初始化向量h0并记录每个时间戳上的ht。

In [ ]:
# 初始化状态张量，用list包裹起来是为了与LSTM, GRU, RNN变种格式统一。
# 状态张量一般为一个矩阵，第一个维度表示batch_size
h0 = [tf.zeros([4, 64])]
x = tf.random.normal([4, 80, 100])
xt = x[:,0,:]

# 构建输入特征f=100，序列长度s=80，状态长度=64的Cell
cell = layers.SimpleRNNCell(64)

# 前向计算
out, h1 = cell(xt, h0)
out.shape, h1[0].shape

In [ ]:
h = h0
# 在序列长度的维度解开输入，即一共有多少个输入的维度解开
for xt in tf.unstack(x, axis = 1):
    
    # 前向计算，每次计算更新状态矩阵h
    out, h = cell(xt, h)

# 最终输出可以聚合每个时间戳上的输出，也可以只取最后时间戳的输出
out = out

#### 多层SimpleRNNCell网络
和卷积神经网络一样，循环神经网络虽然在时间轴上面展开了多次， 但只能算一个网络层。 通过在深度方向堆叠多个 Cell 来实现深层卷积神经网络一样的效果， 大大的提升网络的表达能力。

但是和卷积神经网络动辄几十、 上百的深度层数来比，循环神经网络很容易出现梯度弥散和梯度爆炸到现象，深层的循环神经网络训练起来非常困难，目前常见的循环神经网络模型层数一般控制在十层以内。

In [ ]:
# 初始化输入
x = tf.random.normal([4, 80, 100])

# 取第一个时间戳的输入x0
xt = x[:, 0, :]

# 初始化两层SimpleRNNCell的状态矩阵
h0 = [tf.zeros([4, 64])]
h1 = [tf.zeros([4, 64])]

# 构建两个Cell
cell0 = layers.SimpleRNNCell(64)
cell1 = layers.SimpleRNNCell(64)

# 在时间轴上循环计算多次来实现整个网络的前向运算
for xt in tf.unstack(x, axis = 1):
    out0, h0 = cell0(xt, h0)
    out1, h1 = cell1(out0, h1)

也可以先完成一层在时间戳上的输出，再进行下一层的计算，但两种方法的效果是一样的，因此可以选择简单的方式，即上面的方式进行运算。

#### SimpleRNN层
在实际使用中， 为了简便，我们不希望手动参与循环神经网络内部的计算过程，比如每一层的 状态向量的初始化，以及每一层在时间轴上展开的运算。 通过 SimpleRNN 层高层接口可以非常方便的帮助我们实现此目的。

In [ ]:
# 构建SimpleRNN层，return_sequences如果是True则返回所有时间戳上的输出
# 默认返回最后一个
layer = layers.SimpleRNN(64, return_sequences = False)
x = tf.random.normal([4, 80, 100])
out = layer(x)
out.shape

In [ ]:
# 构建两层SimpleRNN网络
net = tf.keras.Sequential([
    # 因为下一层需要这一层的所有输出，因此return_sequences = True
    layers.SimpleRNN(64, return_sequences = True), 
    layers.SimpleRNN(64)
])
out = net(x)
out.shape

### 5. 情感分类问题实战
#### 预处理

In [ ]:
batchsz = 128
total_words = 10000
max_review_len = 80
embedding_len = 100
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.imdb.load_data(num_words = total_words)

In [ ]:
# 载入word_index
word_index = tf.keras.datasets.imdb.get_word_index(path = 'C:\\Users\\zxndd\\.keras\\datasets\\imdb_word_index.json')

In [ ]:
# 因为word_index中Key是word，而value是ID，我们需要将其进行翻转
word_index = {k:(v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
# 对数据进行还原
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
# 还原一个句子
decode_review(train_x[0])

In [ ]:
# 截断和填充句子，长句保留后面部分，短句在前面填充，使等长
train_x = tf.keras.preprocessing.sequence.pad_sequences(train_x, 
                                                        maxlen = max_review_len
                                                       )
test_x = tf.keras.preprocessing.sequence.pad_sequences(test_x, 
                                                       maxlen = max_review_len
                                                      )

In [ ]:
# 包裹成数据集对象
db_train = tf.data.Dataset.from_tensor_slices((train_x, train_y))
db_test = tf.data.Dataset.from_tensor_slices((test_x, test_y))

# 打乱，并且分
# drop_remainder表示最后一个batch如果数据量达不到batch_size时保留还是抛弃
db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder = True)
db_test = db_test.batch(batchsz, drop_remainder = True)

#### 构建RNN模型

In [ ]:
class MyRNN(tf.keras.Model):
    
    # Cell方式构建多层网络
    def __init__(self, units):
        super(MyRNN, self).__init__()
        
        # 构建初始化状态向量，状态向量长度为units
        self.state0 = [tf.zeros([batchsz, units])]
        self.state1 = [tf.zeros([batchsz, units])]
        
        # 词向量编码，每个词的编码长度=100
        self.embedding = layers.Embedding(total_words, embedding_len, 
                                          input_length = max_review_len)
        
        # 构建两个cell
        self.rnn_cell0 = layers.SimpleRNNCell(units)
        self.rnn_cell1 = layers.SimpleRNNCell(units)
        
        # 构建分类网络，用于将Cell的输出进行分类，2分类
        self.outlayer = layers.Dense(1)
    
    def call(self, inputs, training = None):
        x = inputs # [b, 80]
        x = self.embedding(x) # [b, 80] => [b, 80, 100]
        state0 = self.state0 # 初始化第一层的状态矩阵
        state1 = self.state1 # 初始化第二层的状态矩阵
        for word in tf.unstack(x, axis = 1): # word: [b, 100]
            
            # 开始训练
            out0, state0 = self.rnn_cell0(word, state0, training)
            out1, state1 = self.rnn_cell1(out0, state1, training)
        
        # 通过最后一层
        x = self.outlayer(out1)
        
        # 得到概率
        prob = tf.sigmoid(x)
        return prob

#### 训练与测试

In [ ]:
from tensorflow.keras import optimizers, losses

units = 64 # RNN状态向量长度
epochs = 20

# 构建RNN
model = MyRNN(units)

# 装配
model.compile(optimizer = optimizers.Adam(0.001),
              loss = losses.BinaryCrossentropy(),
              metrics = ['acc'])

# 训练和验证
model.fit(db_train, epochs = epochs, validation_data = db_test)

# 测试
model.evaluate(db_test)

### 6. 梯度弥散和梯度爆炸
连乘运算中，如果权值的最大特征值小于1，会导致求导的元素值接近于0，这种现象叫做梯度弥散，或者梯度消失，如果大于1，会导致求导的元素值爆炸式增长，这种现象叫梯度爆炸。

梯度弥散时，每次梯度更新后参数基本保持不变。

梯度爆炸时，即使更新也使得参数无法达到最优。

#### 梯度裁剪
梯度裁剪与张量限幅非常类似，也是通过将梯度张量的数值或者范数限制在某个较小的区间内，从而将远大于 1 的梯度值减少。

In [2]:
# 梯度裁剪方法1, tf.clip_by_value(min ,max)
a = tf.random.uniform([2, 2])
tf.clip_by_value(a, 0.4, 0.6)

<tf.Tensor: id=10, shape=(2, 2), dtype=float32, numpy=
array([[0.4       , 0.4       ],
       [0.55031085, 0.6       ]], dtype=float32)>

In [8]:
# 通过限制梯度张量W的范数来实现梯度裁剪，tf.clip_by_norm
a = tf.random.uniform([2, 2]) * 5
b = tf.clip_by_norm(a, 5)

# tf.norm计算二范数
tf.norm(a), tf.norm(b)

(<tf.Tensor: id=125, shape=(), dtype=float32, numpy=5.725259>,
 <tf.Tensor: id=130, shape=(), dtype=float32, numpy=4.9999995>)

神经网络的更新方向是由所有参数的梯度张量 W 共同表示的，前两种方式只考虑单个梯度张量的限幅，会出现网络更新方向变更的情况。 如果能够考虑所有参数的梯度 W 的范数，实现等比例的缩放，那么就能很好的限制网络的梯度值，同时不改变网络的更新方向。这就是第三种梯度裁剪的方式。

In [15]:
# 考虑所有参数的梯度W的范数，实现等比例缩放，tf.clip_by_global_norm
w1 = tf.random.normal([3,3])
w2 = tf.random.normal([3,3])

# 计算global_norm
global_norm1 = tf.sqrt(tf.norm(w1) ** 2 + tf.norm(w2) ** 2)

# 根据global_norm和max_norm进行剪裁
(ww1, ww2), global_norm1 = tf.clip_by_global_norm([w1, w2], 2)

# 计算剪裁后的global_norm
global_norm2 = tf.sqrt(tf.norm(ww1) ** 2 + tf.norm(ww2) ** 2)
global_norm1, global_norm2

(<tf.Tensor: id=253, shape=(), dtype=float32, numpy=3.6816232>,
 <tf.Tensor: id=282, shape=(), dtype=float32, numpy=2.0>)

梯度剪裁一般在计算出梯度后，梯度更新之前进行

In [17]:
with tf.GradientTape() as tape:
    logits = model(x) # 前向传播
    loss = criteon(y, logits) # 计算误差

# 计算梯度
grads = tape.gradient(loss, model.trainable_variables)

# 全局梯度剪裁
grads, _ = tf.clip_by_global_norm(grads, 25)

# 更新参数
optimizer.apply_gradients(zip(grads, model_trainable_variables))

#### 梯度弥散
对于梯度弥散现象， 可以通过增大学习率、减少网络深度，添加 Skip Connection 等一系列的措施抑制。

对于深层次的神经网络， 梯度由最末层逐渐向首层传播， 梯度弥散一般更有可能出现在网络的开始数层。 在深度残差网络出现之前， 几十上百层的深层网络训练起来非常困难， 前面数层的网络梯度极容易出现梯度离散现象，从而使得网络长时间得不到更新。深度残差网络较好地克服了梯度弥散现象，从而让神经网络层数达到成百上千。一般来说，减少网络深度可以减轻梯度弥散现象，但是网络层数减少后，网络表达能力也会偏弱，需要用户自行平衡。

### 7. RNN短时记忆
考虑一个长句子：

*今天天气太美好了， 尽管路上发生了一件不愉快的事情， …， 我马上调整好状态， 开开心心地准备迎接美好的一天。*

根据我们的理解， 之所以能够“开开心心地准备迎接美好的一天”， 在于句子最开始处点名了“今天天气太美好了”。可见人类能够很好的理解长句子，但是循环神经网络却不一定能。 研究人员发现，循环神经网络在处理较长的句子时，往往只能够理解有限长度内的信息，而对于位于较长范围类的有用信息往往不能够很好的利用起来。我们把这种现象叫做短时记忆。

能不能够延长这种短时记忆，使得循环神经网络可以有效利用较大范围内的训练数据， 从而提升性能呢？提出了长短时记忆网络(Long Short-Term Memory, LSTM)。

### 8. LSTM原理
LSTM新增了一个状态向量Ct，同时引入了门控机制，通过门控来控制信息的遗忘和刷新。

在 LSTM 中，有两个状态向量𝒄和 ， 其中𝒄作为 LSTM 的内部状态向量，可以理解为LSTM 的内存 Memory，而 表示 LSTM 的输出向量，相对于基础的 RNN 来说， LSTM 把内部 Memory 和输出分开为 2 个变量，同时利用三个门控：输入门(Input Gate)， 遗忘门(Forget Gate)和输出门(Output Gate)来控制内部信息的流动。

门控机制可以理解为控制数据流通量的一种手段， 类比于水阀门： 当水阀门全部打开时，水流畅通无阻地通过； 当水阀门全部关闭时，水流完全被隔断。在 LSTM 中，阀门开和程度利用门控值向量𝒈表示，通过σ(𝒈)激活函数将门控制压缩到[0,1]之间区间， 当σ(𝒈) = 0时，门控全部关闭，输出𝒐 = 0；当σ(𝒈) = 1时，门控全部打开，输出𝒐 = 𝒙。 通过门控机制可以较好地控制数据的流量程度。

#### 遗忘门
遗忘门作用于 LSTM 状态向量𝒄上面，用于控制上一个时间戳的记忆𝒄t−1对当前时间戳的影响。需要添加两个新的参数Wf和bf，可由反向传播算法优化，使用Sigmoid函数激活。

当门控𝒈𝑓 = 1时，遗忘门全部打开， LSTM 接受上一个状态𝒄𝑡−1的所有信息；当门控𝒈𝑓 = 0时，遗忘门关闭， LSTM 直接忽略𝒄𝑡−1， 输出为 0 的向量。

#### 输入门
输入门用于控制 LSTM 对输入的接收程度。 首先通过对当前时间戳的输入𝒙t和上一个时间戳的输出 t−1做非线性变换得到新的输入向量𝒄~𝑡。其中存在两个新的参数Wc和bc，可通过反向传播算法优化，激活函数为tanh。

𝒄~𝑡并不会全部刷新进入 LSTM 的 Memory，而是通过输入门控制接受输入的量。 又引入新的参数Wi和bi，可通过反向传播算法优化。使用Sigmoid函数激活。

输入门控制变量𝑔𝑖决定了 LSTM 对当前时间戳的新输入𝒄~𝑡的接受程度： 当𝑔𝑖 = 0时， LSTM 不接受任何的新输入𝒄~𝑡；当𝑔𝑖 = 1时， LSTM 全部接受新输入𝒄~𝑡。

#### 刷新memory
在经过遗忘门和输入门后，得到的新的状态向量为：ct = gf x c(t-1)(此处为遗忘门) + gi x c~t(此处为输入门)

#### 输出门
LSTM 的内部状态向量𝒄t并不会直接用于输出，这一点和基础的 RNN 不一样。基础的RNN 网络的状态向量 既用于记忆， 又用于输出，所以基础的 RNN 可以理解为状态向量𝒄和输出向量 是同一个对象。 在 LSTM 内部，状态向量并不会全部输出，而是在输出门的作用下有选择地输出。 

引入新的参数Wo和bo，可通过反向传播算法优化，使用sigmoid函数激活。

当输出门𝑔𝑜 = 0时，输出关闭， LSTM 的内部记忆完全被隔断， 无法用作输出，此时输出为 0 的向量；当输出门𝑔𝑜 = 1时，输出完全打开， LSTM 的状态向量𝒄t全部用于输出。 

首先需要将ct经过tanh激活函数后与输入门作用，得到LSTM的输出ht∈[-1, 1]

### 9. LSTM层
在 TensorFlow 中，可以使用 LSTMCell 来手动完成时间戳上面的循环运算，也可以通过 LSTM 层方式一步完成前向运算。

#### LSTMCell

In [23]:
# LSTMCell
x = tf.random.normal([2, 80, 100])
cell = layers.LSTMCell(64) # 创建Cell

# 初始化状态和输出List[h, c]
state = [tf.zeros([2, 64]), tf.zeros([2, 64])]

# 前向运算，更新List[h, c]
for xt in tf.unstack(x, axis = 1):
    out, state = cell(xt, state)

#### LSTM层

In [27]:
net = tf.keras.Sequential([
    layers.LSTM(64, return_sequences = True),
    layers.LSTM(64)
])
out = net(x)

### 10.GRU
LSTM 具有更长的记忆能力， 在大部分序列任务上面都取得了比基础的 RNN 模型更好的性能表现，更重要的是， LSTM 不容易出现梯度弥散现象。 但是 LSTM 相对较复杂， 计算代价较高，模型参数量较大。 于是科学家们尝试简化 LSTM 内部的计算流程， 特别是减少门控数量。 研究发现， 遗忘门是 LSTM 中最重要的门控， 甚至发现只有遗忘门的简化版网络在多个基准数据集上面优于标准 LSTM 网络。 

在众多的简化版 LSTM 中，门控循环网络(Gated Recurrent Unit)是应用最广泛的变种之一。 GRU 把内部状态向量和输出向量合并，统一为状态向量 ， 门控数量也减少到 2 个：复位门和更新门。

#### 复位门

#### 更新门

#### GRU使用方法
在tensorflow中也有Cell方式和层方式实现GRU网络

In [29]:
# GRUCell方式
# 初始化状态向量
h = [tf.zeros([2, 64])]

# 创建GRUCell
cell = layers.GRUCell(64)

for xt in tf.unstack(x, axis = 1):
    out, h = cell(xt, h)
out.shape

TensorShape([2, 64])

In [30]:
# GRU方式
net = tf.keras.Sequential([
    layers.GRU(64, return_sequences = True),
    layers.GRU(64)
])
out = net(x)
out.shape

TensorShape([2, 64])

### 11. LSTM/GRU情感分类问题实战